In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [4]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [5]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [6]:
line = Line([y, z])

In [7]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [8]:
SE6 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f: -c*(5*c+e)/(c+e)})

In [9]:
%time adm_SE6 = SE6.find_admissible_projectivities()

CPU times: user 1.24 s, sys: 95.7 ms, total: 1.34 s
Wall time: 1.54 s


In [10]:
len(adm_SE6)

48

In [11]:
%time simm_SE6 = SE6.find_simmetries(adm_SE6)

CPU times: user 993 ms, sys: 102 ms, total: 1.1 s
Wall time: 1.19 s


In [11]:
len(simm_SE6)

24

In [12]:
G_SE6 = Group(simm_SE6)

In [13]:
G_SE6.is_group()

True

In [14]:
G_SE6.get_order()

24

In [15]:
G_SE6.is_abelian()

False

In [16]:
G_SE6.get_divisors_of_order()

[1, 2, 3, 4, 6, 8, 12, 24]

In [17]:
G_SE6.get_order_of_elements()

[1, 2, 2, 4, 4, 3, 4, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 2, 4, 3, 2, 4, 4, 3]

In [18]:
find_conditions_for_subfamilies(SE6, adm_SE6, simm_SE6)

[]

In [19]:
perms = []
for el in G_SE6:
    perms.append(Permutation(apply_proj_to_eck(el, SE6.eckardt_points)).to_cycles())

In [20]:
gp = PermutationGroup(perms)

In [21]:
gp.order()

24

In [22]:
gp.structure_description()

'S4'

In [23]:
nperms = [[cycle for cycle in perm if len(cycle)>1] for perm in perms]

In [24]:
for gen in gp.minimal_generating_set():
    print(nperms.index(gen.cycle_tuples()))
    print(gen)

13
(1,2,5)(3,6,4)
14
(1,6)(4,5)


In [25]:
gen1 = G_SE6[13]
gen2 = G_SE6[14]

In [26]:
eck_labels = SE6.eckardt_points_labels
eck_labels

[['E1', 'G4', 'F14'],
 ['E2', 'G1', 'F12'],
 ['E2', 'G3', 'F23'],
 ['E3', 'G4', 'F34'],
 ['E4', 'G2', 'F24'],
 ['F13', 'F24', 'F56']]

In [27]:
eck = SE6.eckardt_points
eck

[(0, 0, 0, 1),
 (0, -c^2 - c*e, 2*c^2 - 3*c*e - e^2, 0),
 (0, 1, 1, 0),
 (0, 0, 1, -1),
 (0, -c^2 - c*e, 2*c^2 - 3*c*e - e^2, -3*c^2 + 2*c*e + e^2),
 (0, c^2 + c*e, c^2 + c*e, -3*c^2 + 2*c*e + e^2)]

In [28]:
for i in range(4):
    for j in range(i+1, 5):
        for k in range(j+1, 6):
            point1 = eck[i]
            point2 = eck[j]
            point3 = eck[k]
            if matrix([point1, point2, point3]).minors(3) == [0,0,0,0]:
                print(eck_labels[i], eck_labels[j], eck_labels[k])

['E1', 'G4', 'F14'] ['E2', 'G1', 'F12'] ['E4', 'G2', 'F24']
['E1', 'G4', 'F14'] ['E2', 'G3', 'F23'] ['F13', 'F24', 'F56']
['E2', 'G1', 'F12'] ['E3', 'G4', 'F34'] ['F13', 'F24', 'F56']
['E2', 'G3', 'F23'] ['E3', 'G4', 'F34'] ['E4', 'G2', 'F24']


In [29]:
new_lines = []
lines_plucker = [line.plucker for line in SE6.lines]
for i in range(len(eck)):
    point1 = eck[i]
    for j in range(i+1, len(eck)):
        point2 = eck[j]
        new_line_plucker = Point(matrix([point1, point2]).minors(2))
        if new_line_plucker not in lines_plucker and new_line_plucker not in [line.plucker for line in new_lines]:
            planes = get_planes(new_line_plucker)
            new_lines.append(Line(planes, [point1, point2], new_line_plucker))

In [30]:
new_lines

[(0, 0, 0, 0, c^2 + c*e, -2*c^2 + 3*c*e + e^2) [2*y*c^2 + z*c^2 - 3*y*c*e + z*c*e - y*e^2, -2*x*c^2 + 3*x*c*e + x*e^2],
 (0, 0, 0, 0, 1, 1) [-y + z, x],
 (0, 0, 0, -c^2 - c*e, c^2 + c*e, -2*c^2 + 3*c*e + e^2) [2*y*c^2 + z*c^2 + t*c^2 - 3*y*c*e + z*c*e + t*c*e - y*e^2, -2*x*c^2 + 3*x*c*e + x*e^2],
 (0, 0, 0, 1, -1, -1) [y - z - t, -x]]

In [31]:
for line in new_lines:
    print(line.apply_proj(gen1) in new_lines, line.apply_proj(gen2) in new_lines)

True True
True True
True True
True True


In [32]:
adm_perm = SE6.find_admissible_permutations()
keys = list(SE6.cl_lines.keys())

In [33]:
len(adm_perm)

96

In [34]:
perms_lines = []
for simm in simm_SE6:
    perms_lines.append(Permutation(apply_proj_to_lines(simm, list(SE6.cl_lines.values()))).to_cycles())

In [35]:
Gr = PermutationGroup(perms_lines)

In [36]:
Gr.order()

24

In [37]:
Gr.structure_description()

'S4'

In [38]:
Gr.minimal_generating_set()

[(1,8,13)(2,10,19)(3,27,9)(4,7,15)(5,24,11)(6,23,12)(14,18,22)(16,20,25)(17,21,26),
 (1,3,15,22)(2,19)(4,13,14,18)(5,6)(7,27,9,8)(11,23,21,16)(12,24,20,17)(25,26)]

In [39]:
[el.order() for el in Gr.normal_subgroups()]

[24, 12, 4, 1]

In [40]:
perms_lines = []
for simm in simm_SE6:
    perm = adm_perm[adm_SE6.index(simm)]
    perms_lines.append(Permutation([perm.index(label)+1 for label in keys]).to_cycles())

In [41]:
Gr = PermutationGroup(perms_lines)

In [42]:
Gr.order()

96

In [43]:
Gr.minimal_generating_set()

[(5,6)(11,12)(16,17)(20,21)(23,24)(25,26),
 (1,15)(2,19)(4,13)(7,8)(9,27)(11,24)(12,23)(14,18)(16,20)(17,21),
 (1,8,18,15,4,9)(2,10,19)(3,27,7,22,14,13)(5,16,21,25,24,12)(6,17,20,26,23,11)]

In [44]:
nor_sub = [el for el in Gr.normal_subgroups() if el.order()==4][0]

In [45]:
Gr.quotient(nor_sub).is_isomorphic(SymmetricGroup(4))

True

In [46]:
keys = list(SE6.cl_lines.keys())

In [47]:
print(keys)

['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'F12', 'F13', 'F14', 'F15', 'F16', 'F23', 'F24', 'F25', 'F26', 'F34', 'F35', 'F36', 'F45', 'F46', 'F56']


In [48]:
len(Gr.cosets(nor_sub))

24

In [49]:
gens = nor_sub.minimal_generating_set()

In [50]:
permutations = []
for gen in gens:
    permutations.append([keys[gen.dict()[key]-1] for key in gen.dict()])

In [51]:
print(permutations)

[['F14', 'E2', 'F34', 'G2', 'F45', 'F46', 'F12', 'E4', 'F23', 'G4', 'F25', 'F26', 'G1', 'F56', 'E1', 'F36', 'F35', 'G3', 'F24', 'G5', 'G6', 'E3', 'F16', 'F15', 'E5', 'E6', 'F13'], ['E1', 'E2', 'E3', 'E4', 'E6', 'E5', 'G1', 'G2', 'G3', 'G4', 'G6', 'G5', 'F12', 'F13', 'F14', 'F16', 'F15', 'F23', 'F24', 'F26', 'F25', 'F34', 'F36', 'F35', 'F46', 'F45', 'F56']]


In [52]:
nor_sub.order()

4

In [53]:
nor_sub.minimal_generating_set()

[(1,15)(3,22)(4,8)(5,25)(6,26)(7,13)(9,18)(11,20)(12,21)(14,27)(16,24)(17,23),
 (5,6)(11,12)(16,17)(20,21)(23,24)(25,26)]

In [54]:
nor_sub.as_finitely_presented_group()

Finitely presented group < a, b | a^2, b^2, (b*a)^2 >